In [8]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_csv("SMS_Spam_Ham.csv")

In [10]:
data

,label,message
0,ham,"Hey John, are you free for a quick chat later ..."
1,spam,WINNER! You've won a free iPhone 15! Claim now...
2,ham,Don't forget to pick up milk on your way home.
3,spam,URGENT: Your bank account has been suspended. ...
4,ham,"Okay, sounds good. See you at 7 PM for dinner."
...,...,...
4319,spam,"Your lucky number has won £55,000! Call 0907xx..."
4320,spam,Urgent: Your online identity has been stolen. ...
4321,spam,Get a FREE luxury gaming PC! Just answer a few...
4322,spam,Your bank transaction security has expired. Ge...


In [11]:
data.head()

,label,message
0,ham,"Hey John, are you free for a quick chat later ..."
1,spam,WINNER! You've won a free iPhone 15! Claim now...
2,ham,Don't forget to pick up milk on your way home.
3,spam,URGENT: Your bank account has been suspended. ...
4,ham,"Okay, sounds good. See you at 7 PM for dinner."


In [12]:
data.tail()

,label,message
4319,spam,"Your lucky number has won £55,000! Call 0907xx..."
4320,spam,Urgent: Your online identity has been stolen. ...
4321,spam,Get a FREE luxury gaming PC! Just answer a few...
4322,spam,Your bank transaction security has expired. Ge...
4323,spam,Congratulations! You've received a special rew...


In [13]:
data.shape

(4324, 2)

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    4324 non-null   object
 1   message  4324 non-null   object
dtypes: object(2)
memory usage: 67.7+ KB


In [15]:
data.duplicated().sum()

np.int64(471)

In [16]:
data=data.drop_duplicates(subset=['message'])

In [18]:
data.duplicated().sum()

np.int64(0)

In [19]:
data

,label,message
0,ham,"Hey John, are you free for a quick chat later ..."
1,spam,WINNER! You've won a free iPhone 15! Claim now...
2,ham,Don't forget to pick up milk on your way home.
3,spam,URGENT: Your bank account has been suspended. ...
4,ham,"Okay, sounds good. See you at 7 PM for dinner."
...,...,...
4319,spam,"Your lucky number has won £55,000! Call 0907xx..."
4320,spam,Urgent: Your online identity has been stolen. ...
4321,spam,Get a FREE luxury gaming PC! Just answer a few...
4322,spam,Your bank transaction security has expired. Ge...


In [20]:
print(data['label'].value_counts())

label
ham     2310
spam    1543
Name: count, dtype: int64


In [23]:
from sklearn.preprocessing import LabelEncoder
el=LabelEncoder()
data['label']=el.fit_transform(data['label'])
# Convert categorical labels ('ham', 'spam') into numeric format 
# Required for training classification models with scikit-learn

C:\Users\vodla\AppData\Local\Temp\ipykernel_31132\92712812.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label']=el.fit_transform(data['label'])


In [24]:
data

,label,message
0,0,"Hey John, are you free for a quick chat later ..."
1,1,WINNER! You've won a free iPhone 15! Claim now...
2,0,Don't forget to pick up milk on your way home.
3,1,URGENT: Your bank account has been suspended. ...
4,0,"Okay, sounds good. See you at 7 PM for dinner."
...,...,...
4319,1,"Your lucky number has won £55,000! Call 0907xx..."
4320,1,Urgent: Your online identity has been stolen. ...
4321,1,Get a FREE luxury gaming PC! Just answer a few...
4322,1,Your bank transaction security has expired. Ge...


In [25]:
print(data['label'].value_counts())

label
0    2310
1    1543
Name: count, dtype: int64


In [26]:
# Text preprocessing function to clean raw text data:
import string
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # - Converts text to lowercase
    text = text.lower()  
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # - Removes punctuation and digits
    text = text.translate(str.maketrans('', '', string.punctuation))  
    # - Strips leading/trailing spaces
    text = text.strip() 
    # - Tokenizes and removes stopwords 
    words = text.split()
    # - Applies stemming using PorterStemmer
    words = [ps.stem(word) for word in words if word not in stop_words]  
    return " ".join(words)

data['message'] = data['message'].apply(clean_text)


C:\Users\vodla\AppData\Local\Temp\ipykernel_31132\1093075674.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['message'] = data['message'].apply(clean_text)


In [27]:
data

,label,message
0,0,hey john free quick chat later today
1,1,winner youv free iphon claim bitlyfreeprizexyz
2,0,dont forget pick milk way home
3,1,urgent bank account suspend verifi securebankl...
4,0,okay sound good see pm dinner
...,...,...
4319,1,lucki number £ call xxxxxxx collect
4320,1,urgent onlin ident stolen secur linkcomidentit...
4321,1,get free luxuri game pc answer question linkco...
4322,1,bank transact secur expir gener new code linkc...


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
# Convert text reviews into TF-IDF feature vectors using the top 1,200 most frequent words
vectorizer = TfidfVectorizer(max_features=1200)
# Transform the cleaned 'review' text into a dense NumPy array
X = vectorizer.fit_transform(data['message']).toarray()
# Target labels (already label-encoded earlier)
y = data['label']

In [32]:
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.31378908],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.29014631]], shape=(3853, 1200))

In [33]:
y

0       0
1       1
2       0
3       1
4       0
       ..
4319    1
4320    1
4321    1
4322    1
4323    1
Name: label, Length: 3853, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets (80% train, 20% test)
# Setting random_state ensures the split is reproducible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

In [37]:
# Train a Multinomial Naive Bayes classifier
model1=MultinomialNB()
model1.fit(X_train,y_train)
# Predict on test data
y_pred1=model1.predict(X_test)
# Evaluate performance
acc1=accuracy_score(y_test,y_pred1)
print(f" Naive Bayes Accuracy : {acc1}")
print("Classification Report:")
print(classification_report(y_test,y_pred1))

 Naive Bayes Accuracy : 0.9753566796368353
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       475
           1       0.96      0.98      0.97       296

    accuracy                           0.98       771
   macro avg       0.97      0.98      0.97       771
weighted avg       0.98      0.98      0.98       771



In [38]:
# Train a Logistic Regression classifier
model2= LogisticRegression()
model2.fit(X_train,y_train)
# Predict on test data
y_pred2=model2.predict(X_test)
# Evaluate performance
acc2=accuracy_score(y_test,y_pred2)
print(f" Logistic Regression Accuracy : {acc2}")
print("Classification Report:")
print(classification_report(y_test,y_pred2))

 Logistic Regression Accuracy : 0.9818417639429312
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       475
           1       0.97      0.98      0.98       296

    accuracy                           0.98       771
   macro avg       0.98      0.98      0.98       771
weighted avg       0.98      0.98      0.98       771



In [39]:
# Train a Linear SVM model
model3=LinearSVC()
model3.fit(X_train,y_train)
# Predict on test data
y_pred3=model3.predict(X_test)
# Evaluate performance
acc3=accuracy_score(y_test,y_pred3)
print(f" SVM Accuracy : {acc3}")
print("Classification Report:")
print(classification_report(y_test,y_pred3))

 SVM Accuracy : 0.9922178988326849
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       475
           1       0.99      0.99      0.99       296

    accuracy                           0.99       771
   macro avg       0.99      0.99      0.99       771
weighted avg       0.99      0.99      0.99       771



In [40]:
# Train a Random Forest model
model4=RandomForestClassifier()
model4.fit(X_train,y_train)
# Predict on test data
y_pred4=model4.predict(X_test)
# Evaluate performance
acc4=accuracy_score(y_test,y_pred4)
print(f" Random Forest Accuracy : {acc4}")
print("Classification Report:")
print(classification_report(y_test,y_pred4))

 Random Forest Accuracy : 0.9857328145265889
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       475
           1       0.98      0.98      0.98       296

    accuracy                           0.99       771
   macro avg       0.98      0.99      0.98       771
weighted avg       0.99      0.99      0.99       771



In [41]:
# Train a  KNN model
model5=KNeighborsClassifier()
model5.fit(X_train,y_train)
# Predict on test data
y_pred5=model5.predict(X_test)
# Evaluate performance
acc5=accuracy_score(y_test,y_pred5)
print(f" KNN Accuracy : {acc5}")
print("Classification Report:")
print(classification_report(y_test,y_pred5))

 KNN Accuracy : 0.9844357976653697
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       475
           1       0.99      0.97      0.98       296

    accuracy                           0.98       771
   macro avg       0.99      0.98      0.98       771
weighted avg       0.98      0.98      0.98       771



In [42]:
# Train a Extra Trees Classifier 
model6 = ExtraTreesClassifier()
model6.fit(X_train, y_train)
# Predict on test data
y_pred6 = model6.predict(X_test)
# Evaluate performance
acc6 = accuracy_score(y_test, y_pred6)
print(f"Extra Trees Accuracy: {acc6}")
print("Classification Report:")
print(classification_report(y_test, y_pred6))

Extra Trees Accuracy: 0.9870298313878081
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       475
           1       0.98      0.99      0.98       296

    accuracy                           0.99       771
   macro avg       0.99      0.99      0.99       771
weighted avg       0.99      0.99      0.99       771



In [43]:
# Train a Histogram Gradient Boosting Classifier model
model7 = HistGradientBoostingClassifier()
model7.fit(X_train, y_train)
# Predict on test data
y_pred7 = model7.predict(X_test)
# Evaluate performance
acc7 = accuracy_score(y_test, y_pred7)
print(f"HistGradientBoosting Accuracy: {acc7}")
print("Classification Report:")
print(classification_report(y_test, y_pred7))

HistGradientBoosting Accuracy: 0.9753566796368353
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       475
           1       0.97      0.97      0.97       296

    accuracy                           0.98       771
   macro avg       0.97      0.97      0.97       771
weighted avg       0.98      0.98      0.98       771



In [47]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

results = {
    'Model': ['Naive Bayes', 'Logistic Regression', 'SVM (LinearSVC)', 'Random Forest', 'KNN', 'Extra Trees'],
    'Accuracy': [acc1, acc2, acc3, acc4, acc5, acc6],
    'Precision': [
        precision_score(y_test, y_pred1),
        precision_score(y_test, y_pred2),
        precision_score(y_test, y_pred3),
        precision_score(y_test, y_pred4),
        precision_score(y_test, y_pred5),
        precision_score(y_test, y_pred6)
    ],
    'Recall': [
        recall_score(y_test, y_pred1),
        recall_score(y_test, y_pred2),
        recall_score(y_test, y_pred3),
        recall_score(y_test, y_pred4),
        recall_score(y_test, y_pred5),
        recall_score(y_test, y_pred6)
    ],
    'F1 Score': [
        f1_score(y_test, y_pred1),
        f1_score(y_test, y_pred2),
        f1_score(y_test, y_pred3),
        f1_score(y_test, y_pred4),
        f1_score(y_test, y_pred5),
        f1_score(y_test, y_pred6)
    ]
}


results_df = pd.DataFrame(results)


results_df = results_df.sort_values(by='Accuracy', ascending=False)


print(results_df)


                 Model  Accuracy  Precision    Recall  F1 Score
2      SVM (LinearSVC)  0.992218   0.989865  0.989865  0.989865
5          Extra Trees  0.987030   0.979866  0.986486  0.983165
3        Random Forest  0.985733   0.979798  0.983108  0.981450
4                  KNN  0.984436   0.989655  0.969595  0.979522
1  Logistic Regression  0.981842   0.973154  0.979730  0.976431
0          Naive Bayes  0.975357   0.957096  0.979730  0.968280


In [49]:
import joblib
joblib.dump(model3, 'spam_classifier_model.pkl')

joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [51]:
def predict_spam(text):
    cleaned_text = clean_text(text)  
    vectorized_text = vectorizer.transform([cleaned_text])
    prediction = model3.predict(vectorized_text)
    return "Spam" if prediction[0] == 1 else "Ham"


In [55]:
message = input("Enter a message: ")
print("Prediction:", predict_spam(message))


Prediction: Spam
